In [64]:
import pandas as pd

# Initial list of zipcodes gathered from World Population Review
https://worldpopulationreview.com/zips/colorado

https://worldpopulationreview.com/zips/utah

In [107]:
colorado = pd.read_csv("colorado-zip-codes.csv")
utah = pd.read_csv("utah-zip-codes.csv")
df = pd.concat([colorado, utah], ignore_index=True)
df.rename(columns={"zip": "Zip_Code", "pop": "Total_Population_2024", "city": "City", "county":"County", "state": "State"}, inplace=True)
print(df.shape)
df.head()

(802, 5)


,Zip_Code,City,County,Total_Population_2024,State
0,80134,Parker,Douglas County,75383,CO
1,80013,Aurora,Arapahoe County,73870,CO
2,80015,Aurora,Arapahoe County,72841,CO
3,80016,Aurora,Arapahoe County,64842,CO
4,80219,Denver,Denver County,64696,CO


# Preliminary Data Gathered from Census API

https://www.census.gov/data/developers/data-sets.html

In [108]:
colorado = pd.read_csv("colorado_census_data.csv")
utah = pd.read_csv("utah_census_data.csv")
df1 = pd.concat([colorado, utah], ignore_index=True)
df1 = df1.rename(columns={"zip_code_tabulation_area": "Zip_Code"})
print(df1.shape)
df1.head()

(802, 7)


,NAME,Total_Population,Median_Age,Housing_Units,Median_Gross_Rent,Median_Home_Value,Zip_Code
0,ZCTA5 80134,73165,36.9,26786,1786,508500,80134
1,ZCTA5 80013,75525,35.9,26595,1745,355500,80013
2,ZCTA5 80015,71786,38.2,24706,1865,430100,80015
3,ZCTA5 80016,63013,39.2,22657,1810,593200,80016
4,ZCTA5 80219,67393,33.1,22955,1273,335000,80219


# More In-Depth Migration and Demographic Data from the Census API

In [109]:
colorado = pd.read_csv("colorado_census_demographics.csv")
utah = pd.read_csv("utah_census_demographics.csv")
df2 = pd.concat([colorado, utah], ignore_index=True)
print(df2.shape)
df2.head()

(802, 16)


,ZIP_NAME,Total_Population,Median_Age,White_Population,Black_Population,American_Indian_Alaska_Native_Population,Asian_Population,Hispanic_Latino_Population,Foreign_Born_Population,Total_Moved_Last_Year,Moved_from_Different_State,Moved_from_Abroad,Median_Household_Income,Median_Gross_Rent,Median_Home_Value,Zip_Code
0,ZCTA5 80134,73165,36.9,60219,1784,247,5072,7656,3068,72525,60287,4748,124825,1786,508500,80134
1,ZCTA5 80013,75525,35.9,47640,10225,374,5082,18035,5056,74601,64459,5724,86266,1745,355500,80013
2,ZCTA5 80015,71786,38.2,50292,6280,593,5557,10499,3398,70805,62147,4167,108577,1865,430100,80015
3,ZCTA5 80016,63013,39.2,45592,4124,68,6536,6311,2478,62445,53397,4826,131250,1810,593200,80016
4,ZCTA5 80219,67393,33.1,44053,1105,1009,1371,47267,10822,66542,61580,2529,56441,1273,335000,80219


# Averaged Real Estate Data from the Zillow API

https://www.zillowgroup.com/developers/

In [110]:
colorado = pd.read_csv("average_metrics_per_zip_co.csv")
utah = pd.read_csv("average_metrics_per_zip_utah.csv")
df3 = pd.concat([colorado, utah], ignore_index=True)
df3 = df3.rename(columns={"zip_code": "Zip_Code"})
print(df3.shape)
df3.head()

(802, 6)


,Zip_Code,avg_price,avg_beds,avg_baths,avg_sqft,num_listings
0,80134,734213.780488,3.609756,3.414634,3003.292683,41
1,80013,450697.560976,3.121951,2.365854,1782.170732,41
2,80015,579343.902439,3.756098,2.975610,2576.341463,41
3,80016,878502.195122,3.700000,3.780488,3899.073171,41
4,80219,501832.902439,3.657895,1.820513,1479.871795,41


# Merging all dataframes on Zipcode

In [125]:
common_columns = set(df1.columns) & set(df2.columns)
print("Common columns from Census API to be dropped:", common_columns)


Common columns from Census API to be dropped: {'Total_Population', 'Median_Home_Value', 'Zip_Code', 'Median_Age', 'Median_Gross_Rent'}


In [112]:
common_cols = set(df1.columns) & set(df2.columns)
cols_to_drop = list(common_cols - {'Zip_Code'})

df2_cleaned = df2.drop(columns=cols_to_drop)

merged_df = df1.merge(df2_cleaned, on='Zip_Code').merge(df3, on='Zip_Code').merge(df, on='Zip_Code')
print(merged_df.shape)
merged_df.head()

(802, 27)


,NAME,Total_Population,Median_Age,Housing_Units,Median_Gross_Rent,Median_Home_Value,Zip_Code,ZIP_NAME,White_Population,Black_Population,...,Median_Household_Income,avg_price,avg_beds,avg_baths,avg_sqft,num_listings,City,County,Total_Population_2024,State
0,ZCTA5 80134,73165,36.9,26786,1786,508500,80134,ZCTA5 80134,60219,1784,...,124825,734213.780488,3.609756,3.414634,3003.292683,41,Parker,Douglas County,75383,CO
1,ZCTA5 80013,75525,35.9,26595,1745,355500,80013,ZCTA5 80013,47640,10225,...,86266,450697.560976,3.121951,2.365854,1782.170732,41,Aurora,Arapahoe County,73870,CO
2,ZCTA5 80015,71786,38.2,24706,1865,430100,80015,ZCTA5 80015,50292,6280,...,108577,579343.902439,3.756098,2.975610,2576.341463,41,Aurora,Arapahoe County,72841,CO
3,ZCTA5 80016,63013,39.2,22657,1810,593200,80016,ZCTA5 80016,45592,4124,...,131250,878502.195122,3.700000,3.780488,3899.073171,41,Aurora,Arapahoe County,64842,CO
4,ZCTA5 80219,67393,33.1,22955,1273,335000,80219,ZCTA5 80219,44053,1105,...,56441,501832.902439,3.657895,1.820513,1479.871795,41,Denver,Denver County,64696,CO


In [113]:
merged_df.columns

Index(['NAME', 'Total_Population', 'Median_Age', 'Housing_Units',
       'Median_Gross_Rent', 'Median_Home_Value', 'Zip_Code', 'ZIP_NAME',
       'White_Population', 'Black_Population',
       'American_Indian_Alaska_Native_Population', 'Asian_Population',
       'Hispanic_Latino_Population', 'Foreign_Born_Population',
       'Total_Moved_Last_Year', 'Moved_from_Different_State',
       'Moved_from_Abroad', 'Median_Household_Income', 'avg_price', 'avg_beds',
       'avg_baths', 'avg_sqft', 'num_listings', 'City', 'County',
       'Total_Population_2024', 'State'],
      dtype='object')

In [114]:
merged_df.drop(columns=["NAME", "ZIP_NAME"], inplace=True)

In [115]:
merged_df.columns

Index(['Total_Population', 'Median_Age', 'Housing_Units', 'Median_Gross_Rent',
       'Median_Home_Value', 'Zip_Code', 'White_Population', 'Black_Population',
       'American_Indian_Alaska_Native_Population', 'Asian_Population',
       'Hispanic_Latino_Population', 'Foreign_Born_Population',
       'Total_Moved_Last_Year', 'Moved_from_Different_State',
       'Moved_from_Abroad', 'Median_Household_Income', 'avg_price', 'avg_beds',
       'avg_baths', 'avg_sqft', 'num_listings', 'City', 'County',
       'Total_Population_2024', 'State'],
      dtype='object')

In [116]:
# moving Zip_Code column to the front
zip = merged_df['Zip_Code']
merged_df.drop(labels=['Zip_Code'], axis=1,inplace = True)
merged_df.insert(0, 'Zip_Code', zip)
merged_df.columns

Index(['Zip_Code', 'Total_Population', 'Median_Age', 'Housing_Units',
       'Median_Gross_Rent', 'Median_Home_Value', 'White_Population',
       'Black_Population', 'American_Indian_Alaska_Native_Population',
       'Asian_Population', 'Hispanic_Latino_Population',
       'Foreign_Born_Population', 'Total_Moved_Last_Year',
       'Moved_from_Different_State', 'Moved_from_Abroad',
       'Median_Household_Income', 'avg_price', 'avg_beds', 'avg_baths',
       'avg_sqft', 'num_listings', 'City', 'County', 'Total_Population_2024',
       'State'],
      dtype='object')

In [118]:
df = merged_df
rename_dict = {
    'Zip_Code': 'zip_code',
    'Total_Population': 'total_population',
    'Median_Age': 'median_age',
    'Housing_Units': 'housing_units',
    'Median_Gross_Rent': 'median_gross_rent',
    'Median_Home_Value': 'median_home_value',
    'White_Population': 'white_population',
    'Black_Population': 'black_population',
    'American_Indian_Alaska_Native_Population': 'american_indian_alaska_native_population',
    'Asian_Population': 'asian_population',
    'Hispanic_Latino_Population': 'hispanic_latino_population',
    'Foreign_Born_Population': 'foreign_born_population',
    'Total_Moved_Last_Year': 'total_moved_last_year',
    'Moved_from_Different_State': 'moved_from_different_state',
    'Moved_from_Abroad': 'moved_from_abroad',
    'Median_Household_Income': 'median_household_income',
    'avg_price': 'average_price',
    'avg_beds': 'average_beds',
    'avg_baths': 'average_baths',
    'avg_sqft': 'average_sqft',
    'num_listings': 'num_listings',
    'City': 'city',
    'County': 'county',
    'Total_Population_2024': 'total_population_2024',
    'State': 'state'
}

df.rename(columns=rename_dict, inplace=True)

# Step 2: Define the ideal ordering of the columns.
ideal_order = [
    # Geographic Information
    'state', 'county', 'city', 'zip_code',
    # Population & Demographics
    'total_population', 'total_population_2024', 'median_age',
    # Housing & Economic Metrics
    'housing_units', 'median_gross_rent', 'median_home_value', 'median_household_income',
    # Population Breakdown
    'white_population', 'black_population', 'american_indian_alaska_native_population',
    'asian_population', 'hispanic_latino_population', 'foreign_born_population',
    # Migration Data
    'total_moved_last_year', 'moved_from_different_state', 'moved_from_abroad',
    # Real Estate Listings
    'average_price', 'average_beds', 'average_baths', 'average_sqft', 'num_listings'
]

# Reorder the DataFrame columns
df = df[ideal_order]


# The final aggregate dataframe of Colorado and Utah concerning demographic, migration, and real estate metrics.

## Each row is a Zip Code Area

In [126]:
print(df.shape)
df.head()

(802, 25)


,state,county,city,zip_code,total_population,total_population_2024,median_age,housing_units,median_gross_rent,median_home_value,...,hispanic_latino_population,foreign_born_population,total_moved_last_year,moved_from_different_state,moved_from_abroad,average_price,average_beds,average_baths,average_sqft,num_listings
0,CO,Douglas County,Parker,80134,73165,75383,36.9,26786,1786,508500,...,7656,3068,72525,60287,4748,734213.780488,3.609756,3.414634,3003.292683,41
1,CO,Arapahoe County,Aurora,80013,75525,73870,35.9,26595,1745,355500,...,18035,5056,74601,64459,5724,450697.560976,3.121951,2.365854,1782.170732,41
2,CO,Arapahoe County,Aurora,80015,71786,72841,38.2,24706,1865,430100,...,10499,3398,70805,62147,4167,579343.902439,3.756098,2.975610,2576.341463,41
3,CO,Arapahoe County,Aurora,80016,63013,64842,39.2,22657,1810,593200,...,6311,2478,62445,53397,4826,878502.195122,3.700000,3.780488,3899.073171,41
4,CO,Denver County,Denver,80219,67393,64696,33.1,22955,1273,335000,...,47267,10822,66542,61580,2529,501832.902439,3.657895,1.820513,1479.871795,41


# Saving the dataframe to a csv for further analysis

In [122]:
df.to_csv("Colorado_Utah_Demographics_Migration_Real_Estate.csv", index=False)